In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
from time import time
from random import randint
from IPython.core.display import clear_output
import pandas as pd
import numpy as np
import psycopg2
import requests
import os
import re
import ast

In [2]:
headers = {'User-Agent': 'Mozilla/5.0',
           'From': 'itchyandscratchy@gmail.com'
          }

data_path = '/Users/amanda/Documents/Projects/insight/data/'

In [3]:
url = 'https://noc.esdc.gc.ca/Structure/Hierarchy/1a06c040a8bd4022ac498a80eb089bc1?objectid=%2Fd0IGA6qD8JPRfoj5UCjpg%3D%3D'

joblinks = []
# Main summary page
with requests.get(url, timeout=5, headers=headers) as u:     

    soup = BeautifulSoup(u.content)
    for link in soup.find_all('a'):
        joblinks.append(link.get('href'))
            
linkends = [s for s in joblinks if 'NocProfile' in s]            

In [4]:
print(linkends[0])

/Structure/NocProfile/1b4ca4230a4b4bfab443e71f89863d6c?objectid=NmDF%2Bp5XjPpmMEzWh5irOIJPO4DWI3AoLXAGtO9E%2F%2Fr2xVJHKlh2Vvm4wy6CUIjy


In [5]:
base_url = 'https://noc.esdc.gc.ca'

overview_df = pd.DataFrame({'noc_link':[],
                            'noc':[],
                            'job_group':[],
                            'description':[],
                            'alt_titles':[]})

duties_df = pd.DataFrame({'noc':[],
                          'duties':[],
                          'requirements':[]})

for link in linkends:
    
    try:

        # Main summary page
        url = base_url + link
        with requests.get(url, timeout=8, headers=headers) as u:     
            soup = BeautifulSoup(u.content) 
            #print(soup.prettify())

            # Main website content
            a = soup.find('main', attrs={'class':'container'})

            # Title and overview section
            title = a.find('h2',attrs={'style':'margin-top:25px;margin-bottom:0;'})
            intro = a.find('p', attrs={'class':'mrgn-tp-md'})

            if title is not None and intro is not None:
                noc_summary = title.text.split('–') #Note this is an en-dash, not a hyphen
                noc_text = noc_summary[0]
                noc_code = 'NOC ' + noc_text.lstrip('0').strip()
                job_group = noc_summary[-1].strip()
                print(noc_code + ': ' + job_group)
                job_intro = intro.text.strip()
            else:
                continue

            title_sec = a.find('div',attrs={'id':'IndexTitles'})
            title_list = title_sec.findAll('li')
            alt_titles = [s.text.strip() for s in title_list]

            overview_df = overview_df.append({'noc_link':link,
                                              'noc':noc_code,
                                              'job_group':job_group,
                                              'description':job_intro,
                                              'alt_titles':alt_titles}, ignore_index=True)

            # Duties and requirements
            job_duties = []
            duty_sec = soup.find('h4', text = 'Main duties').findNext('div', attrs={'class':'panel-body'})
            duty_list = duty_sec.findAll('li')
            if duty_list:
                job_duties = [s.text.strip() for s in duty_list]
            else:
                job_duties = 'NaN'

            job_req = []
            req_head = soup.find('h4', text = 'Employment requirements').findNext('ul')
            req_list = req_head.findAll('li')
            if req_list:
                job_reqs = [s.text.strip() for s in req_list]
            else:
                job_reqs = 'NaN'

            duties_df = duties_df.append({'noc':noc_code,
                                          'duties':job_duties,
                                          'requirements':job_reqs}, ignore_index=True)            
            
    except:
        continue
    
 

NOC 11: Legislators
NOC 12: Senior government managers and officials
NOC 13: Senior managers - financial, communications and other business services
NOC 14: Senior managers - health, education, social and community services and membership organizations
NOC 15: Senior managers - trade, broadcasting and other services, n.e.c.
NOC 16: Senior managers - construction, transportation, production and utilities
NOC 111: Financial managers
NOC 112: Human resources managers
NOC 113: Purchasing managers
NOC 114: Other administrative services managers
NOC 121: Insurance, real estate and financial brokerage managers
NOC 122: Banking, credit and other investment managers
NOC 124: Advertising, marketing and public relations managers
NOC 125: Other business services managers
NOC 131: Telecommunication carriers managers
NOC 132: Postal and courier services managers
NOC 211: Engineering managers
NOC 212: Architecture and science managers
NOC 213: Computer and information systems managers
NOC 311: Manage

NOC 3012: Registered nurses and registered psychiatric nurses
NOC 3111: Specialist physicians
NOC 3112: General practitioners and family physicians
NOC 3113: Dentists
NOC 3114: Veterinarians
NOC 3121: Optometrists
NOC 3122: Chiropractors
NOC 3124: Allied primary health practitioners
NOC 3125: Other professional occupations in health diagnosing and treating
NOC 3131: Pharmacists
NOC 3132: Dietitians and nutritionists
NOC 3141: Audiologists and speech-language pathologists
NOC 3142: Physiotherapists
NOC 3143: Occupational therapists
NOC 3144: Other professional occupations in therapy and assessment
NOC 3211: Medical laboratory technologists
NOC 3212: Medical laboratory technicians and pathologists' assistants
NOC 3213: Animal health technologists and veterinary technicians
NOC 3214: Respiratory therapists, clinical perfusionists and cardiopulmonary technologists
NOC 3215: Medical radiation technologists
NOC 3216: Medical sonographers
NOC 3217: Cardiology technologists and electrophysiolo

NOC 7237: Welders and related machine operators
NOC 7241: Electricians (except industrial and power system)
NOC 7242: Industrial electricians
NOC 7243: Power system electricians
NOC 7244: Electrical power line and cable workers
NOC 7245: Telecommunications line and cable workers
NOC 7246: Telecommunications installation and repair workers
NOC 7247: Cable television service and maintenance technicians
NOC 7251: Plumbers
NOC 7252: Steamfitters, pipefitters and sprinkler system installers
NOC 7253: Gas fitters
NOC 7271: Carpenters
NOC 7272: Cabinetmakers
NOC 7281: Bricklayers
NOC 7282: Concrete finishers
NOC 7283: Tilesetters
NOC 7284: Plasterers, drywall installers and finishers and lathers
NOC 7291: Roofers and shinglers
NOC 7292: Glaziers
NOC 7293: Insulators
NOC 7294: Painters and decorators (except interior decorators)
NOC 7295: Floor covering installers
NOC 7301: Contractors and supervisors, mechanic trades
NOC 7302: Contractors and supervisors, heavy equipment operator crews
NOC 73

NOC 9612: Labourers in metal fabrication
NOC 9613: Labourers in chemical products processing and utilities
NOC 9614: Labourers in wood, pulp and paper processing
NOC 9615: Labourers in rubber and plastic products manufacturing
NOC 9616: Labourers in textile processing
NOC 9617: Labourers in food and beverage processing
NOC 9618: Labourers in fish and seafood processing
NOC 9619: Other labourers in processing, manufacturing and utilities


In [6]:
# Save cleaned dataframes to file 

overview_df.to_csv(os.path.join(data_path,'processed','noc-overview.csv'), index=False)


In [52]:
# Try urls to find out if they exist
# Note this is a long process and only generally needs to be run once

url_list = []

for i in range(1,30000):  
    try:
        url = 'https://www.jobbank.gc.ca/marketreport/summary-occupation/{}/ca'.format(i)  
        request = requests.get(url, headers=headers, timeout=5, allow_redirects=False)
        if request.status_code == 200:
            url_list.append(url)
            print(url)
        else:
            print('SKIP')
    except:
        continue

https://www.jobbank.gc.ca/marketreport/summary-occupation/1/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/2/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/3/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/4/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/5/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/6/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/7/ca
SKIP
https://www.jobbank.gc.ca/marketreport/summary-occupation/9/ca


In [51]:
clean_job_urls = [x.replace('https://www.jobbank.gc.ca/marketreport/summary-occupation','') for x in url_list]

In [50]:
# Save url_list to file so I can load it later

file_path = (os.path.join(data_path,'processed','job_url_list.txt'))

with open(file_path, 'w') as filehandle:
    for listitem in clean_job_urls:
        filehandle.write('%s\n' % listitem)